In [21]:
import sys
sys.path.append('../')

import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src.database import MongoDB

from sklearn.preprocessing import OneHotEncoder

pd.options.display.max_columns = 100

%matplotlib inline

In [2]:
def hotencode(df):
    unq, idx = np.unique(df, return_inverse=1)
    col_idx = idx.reshape(df.shape)
    out = np.zeros((len(col_idx),col_idx.max()+1),dtype=int)
    out[np.arange(len(col_idx))[:,None], col_idx] = 1
    return pd.DataFrame(out, columns=unq, index=df.index)

In [3]:
radiant_cols = ['hero_' + str(i) for i in range(5)]
dire_cols = ['hero_' + str(i) for i in range(5, 10)]
hero_cols = ['hero_' + str(i) for i in range(10)]

In [18]:
with open('../data/hero_stats.json', 'r') as f:
    heroes_stats = json.load(f)
    heroes_stats = pd.DataFrame(heroes_stats).set_index('id')
    
with open('../data/hero_data.json', 'r') as f:
    heroes_data = json.load(f)
    heroes_data = pd.DataFrame(heroes_data).set_index('id')
    
assert len(heroes_stats) == len(heroes_data)

no_heroes = len(heroes_stats)

## Overview

In [5]:
df = pd.read_csv('../data/matches_data.csv')
df.shape

(38357, 12)

In [6]:
df.head()

,match_id,hero_0,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,hero_8,hero_9,radiant_win
0,4154000815,83,64,93,99,22,110,32,41,92,11,False
1,4152678603,17,26,99,48,9,70,27,2,1,75,True
2,4152494309,23,120,82,18,26,108,31,7,14,44,False
3,4152425012,121,18,106,21,86,10,82,112,110,14,False
4,4152417604,77,93,45,26,7,119,14,4,106,70,False


### Any null values?

In [11]:
df.isnull().sum()

match_id         0
hero_0           0
hero_1           0
hero_2           0
hero_3           0
hero_4           0
hero_5           0
hero_6           0
hero_7           0
hero_8           0
hero_9           0
radiant_win      0
radiants_team    0
dires_team       0
dtype: int64

### Percentage of radiants winning

In [24]:
df.radiant_win.mean()

0.5379982793232004

It's almost about 50 50, so doesn't seem l

### Are all heroes used in the data matches?

In [7]:
radiants = df[radiant_cols]
print('Number of unique heroes used in radiants:', len(set(radiants.values.ravel())))

dires = df[dire_cols]
print('Number of unique heroes used in dires:', len(set(dires.values.ravel())))

Number of unique heroes used in radiants: 116
Number of unique heroes used in dires: 116


Apparently, all heroes have been used in as both radiants and dires.

### Check if there are any matches with the same sets of heroes that have different outcomes.

In [8]:
df['radiants_team'] = list(map(sorted, df[radiant_cols].values.tolist()))
df['radiants_team'] = df['radiants_team'].map(lambda x: ', '.join(map(str, x)))

df['dires_team'] = list(map(sorted, df[dire_cols].values.tolist()))
df['dires_team'] = df['dires_team'].map(lambda x: ', '.join(map(str, x)))

In [10]:
df.drop_duplicates(subset=['radiants_team', 'dires_team']).shape

(38357, 14)

Since after dropping duplicates based on `radiants_team` and `dires_team`, we still get the original shape of the df which means that all the matches have different combinations of heroes in both radiants and dires.

# Hero data

In [19]:
heroes_data.head()

,attack_type,legs,localized_name,name,primary_attr,roles
id,,,,,,
1,Melee,2,Anti-Mage,npc_dota_hero_antimage,agi,"[Carry, Escape, Nuker]"
2,Melee,2,Axe,npc_dota_hero_axe,str,"[Initiator, Durable, Disabler, Jungler]"
3,Ranged,4,Bane,npc_dota_hero_bane,int,"[Support, Disabler, Nuker, Durable]"
4,Melee,2,Bloodseeker,npc_dota_hero_bloodseeker,agi,"[Carry, Disabler, Jungler, Nuker, Initiator]"
5,Ranged,2,Crystal Maiden,npc_dota_hero_crystal_maiden,int,"[Support, Disabler, Nuker, Jungler]"


# Hero Status

In [22]:
heroes_stats

,1_pick,1_win,2_pick,2_win,3_pick,3_win,4_pick,4_win,5_pick,5_win,6_pick,6_win,7_pick,7_win,8_pick,8_win,agi_gain,attack_range,attack_rate,attack_type,base_agi,base_armor,base_attack_max,base_attack_min,base_health,base_health_regen,base_int,base_mana,base_mana_regen,base_mr,base_str,cm_enabled,hero_id,icon,img,int_gain,legs,localized_name,move_speed,name,primary_attr,pro_ban,pro_pick,pro_win,projectile_speed,roles,str_gain,turn_rate
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3819,1861,17546,8841,38115,19515,54088,28335,38746,20537,10384,5525,1565,824,149,83,2.8,150,1.4,Melee,22,-1.0,33,29,200,1.75,12,75,0.9,25,23,True,1.0,/apps/dota2/images/heroes/antimage_icon.png,/apps/dota2/images/heroes/antimage_full.png?,1.80,2,Anti-Mage,310,npc_dota_hero_antimage,agi,20.0,17.0,10.0,0,"[Carry, Escape, Nuker]",1.30,0.5
2,6126,3331,26952,14620,51597,27905,68232,36490,48797,25890,14492,7616,2839,1425,339,166,2.2,150,1.7,Melee,20,-2.0,28,24,200,4.25,18,75,0.9,25,25,True,2.0,/apps/dota2/images/heroes/axe_icon.png,/apps/dota2/images/heroes/axe_full.png?,1.60,2,Axe,290,npc_dota_hero_axe,str,43.0,57.0,33.0,900,"[Initiator, Durable, Disabler, Jungler]",2.80,0.6
3,712,337,2982,1324,6645,2902,10286,4727,9229,4308,3691,1819,1186,577,319,174,2.4,400,1.7,Ranged,23,1.0,41,35,200,1.50,23,75,0.9,25,23,True,3.0,/apps/dota2/images/heroes/bane_icon.png,/apps/dota2/images/heroes/bane_full.png?,2.40,4,Bane,310,npc_dota_hero_bane,int,48.0,87.0,44.0,900,"[Support, Disabler, Nuker, Durable]",2.40,0.6
4,6735,3505,24397,12680,41077,21296,50922,26689,35503,18452,12009,6279,2948,1555,514,276,3.0,150,1.7,Melee,24,0.0,39,33,200,1.50,18,75,0.9,25,24,True,4.0,/apps/dota2/images/heroes/bloodseeker_icon.png,/apps/dota2/images/heroes/bloodseeker_full.png?,1.70,2,Bloodseeker,285,npc_dota_hero_bloodseeker,agi,89.0,53.0,26.0,900,"[Carry, Disabler, Jungler, Nuker, Initiator]",2.70,0.5
5,4001,2191,18742,10161,39066,21039,58829,31605,48403,25501,17035,8948,3931,2096,675,368,1.6,600,1.7,Ranged,16,0.0,32,26,200,1.50,14,75,0.9,25,18,True,5.0,/apps/dota2/images/heroes/crystal_maiden_icon.png,/apps/dota2/images/heroes/crystal_maiden_full....,2.90,2,Crystal Maiden,275,npc_dota_hero_crystal_maiden,int,5.0,67.0,41.0,900,"[Support, Disabler, Nuker, Jungler]",2.00,0.5
6,6701,3533,23290,12135,39264,20452,47885,25241,33876,18131,11348,6198,2499,1334,413,179,2.2,625,1.7,Ranged,19,-3.0,36,25,200,1.75,15,75,0.9,25,17,True,6.0,/apps/dota2/images/heroes/drow_ranger_icon.png,/apps/dota2/images/heroes/drow_ranger_full.png?,1.40,2,Drow Ranger,285,npc_dota_hero_drow_ranger,agi,166.0,62.0,40.0,1250,"[Carry, Disabler, Pusher]",1.90,0.7
7,4074,2058,21654,11029,53020,26980,89845,45560,81689,41915,30154,15526,7047,3590,1350,729,1.4,150,1.7,Melee,12,1.0,37,27,200,2.50,16,75,0.9,25,22,True,7.0,/apps/dota2/images/heroes/earthshaker_icon.png,/apps/dota2/images/heroes/earthshaker_full.png?,1.80,2,Earthshaker,310,npc_dota_hero_earthshaker,str,149.0,206.0,109.0,0,"[Support, Initiator, Disabler, Nuker]",3.20,0.9
8,4456,2222,20158,10081,44101,22290,66774,34199,52056,26565,15244,7681,2494,1194,238,115,2.4,150,1.4,Melee,26,1.0,26,22,200,2.00,14,75,0.9,25,21,True,8.0,/apps/dota2/images/heroes/juggernaut_icon.png,/apps/dota2/images/heroes/juggernaut_full.png?,1.40,2,Juggernaut,295,npc_dota_hero_juggernaut,agi,10.0,22.0,7.0,0,"[Carry, Pusher, Escape]",2.20,0.6
9,2338,1162,13426,6908,37847,19297,67494,34451,58438,29374,18590,9014,3507,1762,620,307,3.2,630,1.7,Ranged,18,-1.0,30,25,200,1.50,17,75,0.9,25,18,True,9.0,/apps/dota2/images/heroes/mirana_icon.png,/apps/dota2/images/heroes/mirana_full.png?,1.65,2,Mirana,295,npc_dota_hero_mirana,agi,70.0,164.0,89.0,900,"[Carry, Support, Escape, Nuker, Disabler]",2.15,0.5
